# Project 03: Prediction and Inference on Vehicle Stops

### Checkpoint #1 Due Date: Friday March 8, 11:59 PM

### Final Due Date: Friday March 15, 11:59 PM
 
---

**The first checkpoint requires turning in questions 1,2,3.**

The project will be auto-graded graded like the HW assignments. The free response questions are all **optional**, but will be good practice for questions on the final Exam.

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
%matplotlib inline
import seaborn as sns

In [3]:
%load_ext autoreload
%autoreload 2

In [7]:
import project03 as proj

# Vehicle Stops Data

This project will focus on prediction and inference on the vehicle stops data. As you have already spent time understanding and cleaning this dataset, we will be using an *already cleaned* stops dataset. This dataset consists of vehicle stops between 2015-01-01 and 2017-12-31 inclusive, with messy values handled similarly to as we did in project 1. The data cleaning code used to create the data can be found in the file `cleaning.py`; this file is included for transparency (you do not need to use it) and to understand what's been done to the data you're familiar with.

As this dataset is a little different from those you've analyzed in the past, you're encourage to do your own EDA. In particular, since the dataset spans 3 years, the time-series analyses will look quite different.

In [4]:
stops_fp = os.path.join('data', 'vehicle_stops_datasd.csv')
stops = pd.read_csv(stops_fp)

In [5]:
len(stops)

102879

# Predictive Policing

---

## Introduction

The industry of so-called "data-driven predictive policing" has been growing in recent years. People and organizations of all motivations have been pushing for better use of data to help police communities across the US. Some of the motivations for such technological approaches to polices are:

1. Create a "more fair" police force using algorithms,
2. Better allocating limited resources for cash-deprived cities,
3. Making policing safer by using advanced diagnostic techniques.

A few specific examples of such projects are:
1. Predicting whether a given traffic stop will need "further investigation" (a search, an arrest).
2. Predicting how many traffic stops there will be in a given service area, during a given month-day-hour. With this knowledge, a police department will know how many officers to deploy to which areas for shifts throughout the year.

All of these approaches start with good intentions. However, it can be seem like you're creating a good solution, while in fact your solution only achieves mediocre performance with unintended consequences. Being able to bring a critical eye to these models is of utmost importance. 

In particular, understanding *why* the predictions are what they are, is vital in the context of fundamental rights. For example:
* If you were pulled over and an algorithm decides you should be searched (incorrectly), wouldn't you like to know why it decided on that action?
* If a "resource allocation" algorithm decides to place police outside your home to give traffic tickets, wouldn't you like to know it's done for appropriate reasons? (e.g. there have been a disproportionate number of accidents at that location).

There has been vigorous discussion in the media on this subject. A few introductory articles:
* [predictive policing and civil liberties](https://slate.com/technology/2015/11/using-data-science-for-predictive-policing-has-serious-civil-liberties-drawbacks.html)
* [Fairness in policing with data](http://www.bigdatadialog.com/fairness/mne5eiqeykqj4ksyley9uo72k41xwc)
* [Big Data in policing (FBI)](https://leb.fbi.gov/articles/featured-articles/predictive-policing-using-technology-to-reduce-crime)
* [Preemptive contact](https://www.nytimes.com/2015/09/25/us/police-program-aims-to-pinpoint-those-most-likely-to-commit-crimes.html?_r=0)

A tricky, but vital, issue to deal with in this problem is that of "feedback loops". That is, when you take actions based on a model's suggestion, you will naturally have more information about what occurred from that action (as opposed to a non-action). This leads to confirmation bias. When you don't have feedback about your failure, your models behavior only gets reinforced, even if it's (in)actions are incorrect on a large portion of the population.

---

## Predicting a Search

In this project we will train a model to predict whether a given traffic stop results in a search. The context for how such a model is used might be as follows:

* A police officer pulls over a driver and collects the driver's information (e.g. the data seen in our dataset, plus likely more). The officer then asks the model to "diagnose" the situation -- i.e. whether they should search the vehicle.

You will approach this as a classification problem: train a classifier to predict whether a given traffic stop resulted in a search. You will then analyze the resulting model's performance and the quality of the output (e.g. for 'fairness').

### EDA and analysis of the target

The target we'll use in our classifier is the `searched` column. Before beginning training a model, you have to assess the quality of the target. The two relevant properties of the search column are:
1. The outcome we are trying to predict is relatively rare. This makes it harder find the patterns to learn. Verify for yourself the proportion of traffic stops that results in a search.


In [1]:
# find search rate


2. The target column has NULL values that we either need to drop or impute. For either of these ways forward, we need to understand the consequences of our actions (e.g. did we introduce bias? in what way?).

Similar to what we saw in project 01, the `searched` field is likely 'Missing at Random', conditional on `service_area` and `race_code`. The dependence on `race_code` is largely attributable to White (W) and Hispanic (H) values, which is in-turn correlated to the dependence on `service_area` (and the proximity to county borders and border control checkpoints).

In `cleaning.py`, the function `impute` performs imputation by drawing from the distribution of `searched` conditional on `service_area`. The results of performing this imputation are given in the dataset `data/stops_for_prediction.csv`.

Since there is randomness involved in the imputation, the "right thing" would be to perform our prediction analyses multiple times for *multiple imputations* and analyze the variance of our predictions.

**You should be able to do both: verify MAR, as well as create the imputation function yourself, if asked!**

Now read in the imputed `searched` column and verify the proportion of traffic stops that results in a search.

In [64]:
searched_fp = os.path.join('data', 'imputed_searched.csv')
searched = pd.read_csv(searched_fp, index_col=0, names=['searched'], squeeze=True)

In [ ]:
# find search rate


### Creating a feature generating pipeline

A tricky aspect of this dataset is that *most of the attributes are categorical*. To start, we will create a `sklearn` Pipeline that generates a number of obvious, generic features for the dataset. That is, we will

1. One-hot encode the nominal columns,
2. Create a smoothing feature on `subject_age`, and
2. Use other ordinal and quantitative columns as they appear in the dataset.

**Question 01**

Create a function that one-hot encodes the nominal columns (`stop_cause`, `service_area`, `subject_race`, `subject_sex`, `sd_resident`) -- that is, create a function `create_cat_feats` of zero-variables that outputs a pipeline that transforms an array consisting of the nominal columns and outputs an array of one-hot encoded values. These nominal columns have null values; take the approach of treating null values as just another category (replace it with the string `NULL` using `sklearn.impute.SimpleImputer`).


**Question 02**

Recall that the `subject_age` column contained noise that led us to believe that the data is only approximately accurate up to ~5 year intervals. Next you will make transformer that transforms the `subject_age` column as follows:
* If an age is missing, replace it with the mean (**this isn't good practice**, but part of what we're doing is building a naive, baseline model).
* Given an age, if it's not divisible by 5, round it to the nearest integer divisible by 5.

*Note:* We could also use smoothing practices here instead.

Create a function `create_age_feats` that takes in a dataframe of one column (`subject_age`) and outputs an unfitted pipeline that transforms the column as described above.

### Creating a baseline model

**Question 03**

Create a modeling pipeline of that takes in `stops`/`searched` as the data/target and outputs an *unfit* model that, once fit, predicts whether the given traffic stop will result in a search. The baseline model should be a `LogisticRegression` model with default parameters.

That is, your pipeline should consist of a feature transformation piece and an model piece. The feature transformation piece takes in the `stops` data and transforms the data into the following features:
* The one-hot encoding from question 1 (reuse your code!)
* The smoothed age from question 2 (reuse your code!)
* The hour and dayofweek (untouched); use `FunctionTransformer`, or a keyword argument in `ColumnTransformer`.

The features are put together using `ColumnTransformer` and combined with a `LogisticRegression` estimator, just like in lecture and on homework 8.

Specifically:
* Create a function `baseline_model` of zero variables that creates and returns this pipeline.
* Create a function `train_test_acc` that takes in the unfit baseline model above, the data `stops` and `searched` and returns a tuple that contains the accuracy on the training set and the test set.


**Question 04**

Wow -- that accuracy seems pretty good for a baseline model! However, for how many vehicle stops did the model actually decide to search the vehicle? 

* What is the accuracy of 'a constant model' that decides never to execute a search? (i.e. always predicts zero). Create a function called `constant_model_acc` which takes in the target `searched` and outputs the accuracy of the constant model.

* Now, create a function `model_outcomes` that takes in two columns `predictions` and `target`, and returns a binary dataframe of outcomes (true/false positive, true/false negative) for each row. Each row will have exactly one '1' and three '0's. The output of this function is useful for exploring attributes of the training data which may have lead to a stop being a FN/FP. (FN is False Negative, FP is False Positive).

* Create a function `metrics` that takes in two columns `predictions` and `target`, and returns a Series of evaluation metrics: 
    - accuracy, recall, specificity, precision, false-negative rate, false-positive rate, false discovery rate, and F1-score.
    
*Question for studying:* Which of these seems most important for the given problem (given being searched is a rare event)? What is a false positive vs. a false negative in this situation? Is one worse than the other (this is asking for your opinion). Is there bias toward one of these outcomes as defined by the labeling of our data. 

In reality we would like to ask these questions in terms of the data generating process, not the data itself. 
* Police sometimes search people unnecessarily -- should that be a false negative in our model?
* Police sometimes do not search people that should be searched -- should that be a false positive in our model?
    
In fact *our model is just learning the patterns of prior police behavior*. Our model *does nothing* to predict who **should** be pulled over (which is a causal question!).    

### Improving our model features: conditional additive smoothing

**Question 05**

Now that we know that our model isn't much better than a constant model, we can work on improving it! Since our model doesn't have much more than a few one-hot encoded nominal features, let's try to put a little more work into feature engineering.

Recall from Homework 4 that *conditional additive smoothing* interpolates between an observed empirical rate of occurrence and a baseline rate. Here we want to generate features that encode a categorical feature that has some *prior knowledge* of the search rate.

Recall the formula:

$$ p_i = \frac{b_i + \alpha\cdot(b/N)}{x_i + \alpha} \qquad {\rm (smoothed)}$$

For example, if we're transforming the `subject_sex` column, we'll be encoding the prior search rate by gender (in the training set). Thus:
* `M` will transform to approximately the search rate for men (since the number of `M` is large),
* `F` will transform to approximately the search rate for women (since the number of `F` is large),
* `X` will transform to approximate the default search rate 0.04 (since the number of `X` is small).

Since one has to be very careful not to leak information from the test set into your model training, we will create this custom feature in our own custom `sklearn` estimator.

`project03.py` contains starter code for creating your own estimator called `AdditiveSmoother` that implements the following methods:
* `AdditiveSmoother.fit(X, y)` returns the estimator itself with:
    - an attribute called `self.srate` which contains the overall search rate in `X` (a float).
    - an attribute called `self.smdists` which is a dictionary of dictionaries. `self.smdists` stands for smoothed distributions, is keyed by the columns of `X`, and contains a dictionary `{val:rate}` where `val` are the distinct values of the column and `rate` is the search rate of those stops with that value.
* `AdditiveSmoother.transform(X)` returns the encoded smoothed search rates for each column in `X`. If a value occurs in a column of `X` that isn't contained in `self.smdists`, then it should return the overall search rate (`self.srate`).

Fill in the starter code, so that:
1. `addsmooth = AdditiveSmoother(alpha=10)` instantiates an additive smoothing feature.
2. `addsmooth.fit(X, y)` creates the dictionaries containing the information necessary to apply the additive smoothing formula to the columns of `X`.
3. `addsmooth.transform(X)` applies the additive smoothing formula on the columns of `X`.

You will use this transformer to create features in the next question.

*Note*: Refer back to HW04 for more context on the formula and meaning of the transformation.

### Improving our model pipeline

**Question 06**

Next, create a final model using the `ConditionalAdditiveSmoother` from above, instead of one-hot-encoding to train a more complex model. Additionally, try your luck using a `RandomForestClassifier`, which takes the majority vote of many decision trees (by default 10). You have many parameters you can tune; here are a few to pay particular attention to:
* `alpha` in the additive smoothing,
* `n_estimators` in the `RandomForestClassifier` (number of trees)
* Other tree-based parameters in the `RandomForestClassifier`.

Feel free to try other classifiers, and feel free to make other features; try to get an F1-Score near 0.05 if you can!

Create your model in a function of zero variables called `final_model` that returns an *unfitted* pipeline containing your model.

*Note*: Remember to hold out a test-set that remains untouched until the *very* end. The F1-Score should be evaluated on this test-set.

### Evaluating your model

Now that you've trained you model to predict when a vehicle stop leads to a search, it's time understand the performance of your model on a more detailed level. First, rerun your evaluation functions for question 04 on the predictions of the output from your new model. As always, run your evaluations on a test set!

**Question 07**

Now we would like to see how your predictions behave on different groups of people. Does your model treat different groups of people similarly? Keep in mind that the training data doesn't always (usually) treat different groups similarly, so we'll be most interested in the differences between our model output and the training data.

First given a (discrete) column `col`, we'd like to compare the actual search rates against the predicted search rates across different values of the column. For example, are men and women searched at similar rates by the model vs in reality?

* Create a function `compare_search_rates` that takes in `stops` data, `prediction` data from the model output, and a column name `col`. It should return a dataframe:
    - indexed by the distinct values of `col`,
    - with a column `searched` that gives the actual search rates across `col`,
    - with a column `predicted` that gives the predicted search rates across `col`.

*Note: be sure to interpret the results you see here! Is one column larger than the other? what are the sizes of the discrepancies? Try this on Gender, Race, Service Area, for example.*

Second, we'd like to understand how the model performs generally across groups. For example, one form of 'fairness' is to ask whether, when a model makes mistakes, whether it makes them at the same rate for different groups of people. To approach these questions broadly:

* Create a function `compare_metrics` that takes in `stops` data, `prediction` data from model output, and a column name `col`. It should return a dataframe:
    - indexed by the distinct values of col,
    - with columns given by the values of each metric listed in the function `metrics` in Question 04, applied to the group given by that value of column.

* For example, if `col` is `subject_sex`, then one row of the output would contain the accuracy, precision, recall, etc for Males in the stops data.


*Note:* Where do you see a big discrepancy? Which metric do you think would be worst to have a large discrepancy in? (in terms of fairness?)

### Optional further exploration

* Use your `model_outcomes` function to do EDA on the inaccurate predictions given by your model. What attributes are more correlated with false positives? false negatives? Who are the people receiving the incorrect predictions?

### Congratulations! You are done with the project. Don't forget to submit to Gradescope!